- <a href="#1">1. Tweepy Notebook</a>
- <a href="#2">2. AWS Comprehend</a>

# <a id='1'> Tweepy Notebook </a>

Notebook will provide an example of how to use the Tweepy package to read tweets, do some simple processing and then load them into pandas.

The first thing you need to do is log into **Twitter** and create an application.

[Twitter Apps](https://apps.twitter.com)

Select the **Create New App** button

and fill out the application information.

You will ultimately need the following pieces of information:

- access_token
- access_token_secret
- consumer_key
- consumer_secret


## <a id='2'>AWS Comprehend Sentiment Analysis</a>

After collecting all of the tweets and example of using the AWS Sentiment analysis API is demonstrated.



In [1]:
import tweepy
import json
import re
import pandas as pd
from config import Config as cfg

In [2]:
access_token = cfg.access_token
access_token_secret = cfg.access_token_secret
consumer_key = cfg.consumer_key
consumer_secret = cfg.consumer_secret

tweet_file_name = 'tweets.txt'
file_mode = 'a'

Create a tweepy OAuthHandler, to be used when we stream the tweets.

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)


In [25]:
class MyStreamListener(tweepy.StreamListener):
    def __init__(self, api=None, file_name='tweets.txt', mode='a', max_tweets=300):
        super(MyStreamListener, self).__init__()
        self.num_tweets = 0
        self.file = open(file_name, mode)
        self.max_tweets = max_tweets

    def on_status(self, status):
        tweet = status._json
        self.file.write( json.dumps(tweet) + '\n' )
        self.num_tweets += 1
        if self.num_tweets < self.max_tweets:
            return True
        else:
            return False
        self.file.close()

    def on_error(self, status):
        print(status)


Select a filter word or words.  Becareful to pick a set of filter words that will give you hits on tweets quickly, else the stream will wait until it has enough tweets before it exits.

In [26]:
filter_words=['trump']

In [27]:
l = MyStreamListener()
stream = tweepy.Stream(auth, l, file_name=tweet_file_name, mode=file_mode, max_tweets=100)

# filters twitter streams to capture data by keywords
stream.filter(track=filter_words)


### Post process the collected tweets

There is clearly a number of things we can do to clean up the tweet text.  I wanted this to be a simple example of some minimal post processing

In [28]:
def word_in_text(word, tweet):
    word = word.lower()
    text = tweet.lower()
    match = re.search(word, text)

    if match:
        return True
    return False


In [29]:
tweets_data = []
tweets_with_words = ['russia', 'SeanHannity', 'Mueller', 'clinton']
with open(tweet_file_name, 'r') as tweets_file:
    for line in tweets_file:
        tweet = json.loads(line)
        user = tweet['user']
        tweet['user_id'] = user['id']
        tweet['screen_name'] = user['screen_name']
        tweet['len'] = len(tweet['text'])
        if tweet['text'].startswith('RT '):
            tweet['retweet'] = 1
            tweet['text'] = tweet['text'].replace('RT ', '', 1)
        else:
            tweet['retweet'] = 0
        for word in tweets_with_words:
            tweet[word] = 1 if word_in_text(word, tweet['text']) else 0
        tweets_data.append(tweet)


In [30]:
columns=['text', 'screen_name', 'len', 'retweet']
columns.extend(tweets_with_words)
df = pd.DataFrame(tweets_data, columns=columns)

print(df.head(20))
print(df.shape)

                                                 text     screen_name  len  \
0             @AlamoOnTheRise https://t.co/ukp07vXGqf        ne1for23   39   
1                     So true https://t.co/Vq9ayIAcAn        jend0315   31   
2   @sumariumcom: Cabello sobre ausencia de Trump ...   REACCI0NARIAA  139   
3   @TrumpTrainMRA4: @facebook \n@DiamondandSilk \...     Crowntiptoe  140   
4   @page88: Thread. Don’t think this overstates m...    JadeJensen29   80   
5   @SpeakerRyan @SenateMajLdr #ProtectMueller #Pr...      Terryg1979   85   
6   @funder: Trump sending troops to the border, i...   IamBrooklyn31  140   
7   @AnnCoulter: Mueller is DYING to get fired.  T...          SPeek1  139   
8   Yup.\nTrain wreck coming.\n#Resist\n#Resign\n#...       titeman50   73   
9   @drscott_atlanta: #FBI raids @realDonaldTrump’...       Jdflygirl  140   
10  @MSNBC: The New York Times reports special cou...     hani_azer11  140   
11  @CoreyLMJones: Mark Zuckerberg said that Faceb...  Mikeypoos

In [31]:
df.to_csv('./data/tweets.txt')

# AWS Comprehend Sentiment Analysis

This section assumes you have an AWS account, and you have created an IAM user and generated the access tokens.

## AWS Comprehend API

To use the comprehend API, and in particular the sentiment analysis api:

- *pip install awscli*
- *aws configure*
- supply your users access key and secret key which you get from the IAM console

See the following documents:
- [Python Boto3 API](https://boto3.readthedocs.io/en/latest/guide/quickstart.html)

- [Amazon Comprehend Documents](https://docs.aws.amazon.com/comprehend/latest/dg/what-is.html)

The above will setup the configuration in the default profile.


In [32]:
tweets_df = pd.read_csv('./data/tweets.txt')

In [33]:
import boto3
import json

In [34]:
comprehend = boto3.client(service_name='comprehend', region_name='us-east-1')

For each of the tweets, call the sentiment API and add to the data frame the resulting sentimment, and the individual probabilities of the different sentiments.

In [36]:
for tweet in tweets_df['text']:
    response = comprehend.detect_sentiment(Text=tweet, LanguageCode='en')
    sentiment = response['Sentiment']
    sentiment_mixed = response['SentimentScore']['Mixed']
    sentiment_negative = response['SentimentScore']['Negative']
    sentiment_neutral = response['SentimentScore']['Neutral']
    sentiment_positive = response['SentimentScore']['Positive']
    tweets_df['sentiment'] = sentiment
    tweets_df['sentiment_mixed'] = sentiment_mixed
    tweets_df['sentiment_negative'] = sentiment_negative
    tweets_df['sentiment_neutral'] = sentiment_neutral
    tweets_df['sentiment_positive'] = sentiment_positive
    
    print(f"[{sentiment}\n{tweet}\n]")


[NEUTRAL
@AlamoOnTheRise https://t.co/ukp07vXGqf
]
[NEUTRAL
So true https://t.co/Vq9ayIAcAn
]
[NEUTRAL
@sumariumcom: Cabello sobre ausencia de Trump en Lima: “Que tristeza para los perritos que esperaban pasarle la lengua por los zapatos”…
]
[NEUTRAL
@TrumpTrainMRA4: @facebook 
@DiamondandSilk 
Mark Zuckerberg Declares Two Black Trump Chicks 
“ UnSafe for His Community “
🤪Liberalism i…
]
[NEUTRAL
@page88: Thread. Don’t think this overstates matters. https://t.co/EpKFaUBi62
]
[NEUTRAL
@SpeakerRyan @SenateMajLdr #ProtectMueller #ProtectRosenstein https://t.co/z9ihMbnVAM
]
[NEUTRAL
@funder: Trump sending troops to the border, is literally his way of making the military pay for the wall. Because the military themselv…
]
[NEUTRAL
@AnnCoulter: Mueller is DYING to get fired.  Then he'll be a martyr! A hero!  Otherwise, he's just a loser who couldn't find any crimes…
]
[NEUTRAL
Yup.
Train wreck coming.
#Resist
#Resign
#Impeach https://t.co/SGQMXZKaKR
]
[NEUTRAL
@drscott_atlanta: #FBI raids @re

[NEUTRAL
@tribelaw: Anyone advising Trump he has power to remove Mueller better read  In re Hennen (1839), reaff’d by CJ Taft (1926) and CJ Rober…
]
[NEUTRAL
@CNN: Special counsel Robert Mueller’s investigators met with members of President Trump’s legal team on the same day that the FBI was l…
]
[NEUTRAL
@NormEisen: I have analyzed thousands of conflicts as a gov ethicist, &amp; argument WH is floating that Rosenstein has 1 b/c he is a potent…
]
[NEUTRAL
@SaysHummingbird: 300,000 Americans vow to protest across all 50 states if Trump fires Mueller.

Retweet if you are one of them.

#Firep…
]
[NEUTRAL
Trump Hotel to Employees: Dont Hire Relatives https://t.co/StGr0Xn3kr
]
[NEUTRAL
@mog7546: #Maddow unveils NEW EVIDENCE REFUTING #Trump's claim that James Comey is lying about their conversations 

https://t.co/cbQdsp…
]
[NEUTRAL
@gabrielsherman: As Trump fumes about Cohen raid, Bolton and Kudlow are clashing with Kelly. My latest:
https://t.co/QER1fpfyKJ
]
[NEUTRAL
@pjbruno79: It's amazi

[NEUTRAL
@MikeTokes @jimmykimmel Mike, is that vid even legit⁉️ It has to be 20+ years old. trump sheep 🐑really love to go d… https://t.co/o4frm3C18l
]
[NEUTRAL
@RVAwonk: Wow. @maddow obtained Dana Boente's handwritten, contemporaneous notes from his conversations with James Comey. And those note…
]
[NEUTRAL
@SethAbramson: PLEASE RT If you strenuously and unambiguously object to Trump firing Deputy Attorney General Rosenstein or Special Couns…
]
[NEUTRAL
@RVAwonk: While questioning #Zuckerberg, Senators Tom Tillis &amp; Chuck Grassley both falsely claimed that the Obama campaign used the same…
]
[NEUTRAL
Schumer to Trump: Don't even think about firing Mueller https://t.co/eahaKhilyF via @YouTube
]
[NEUTRAL
Trump mulling firing Rosenstein: report https://t.co/Temp2yQDdq https://t.co/573Lpuqr8u
]
[NEUTRAL
@emptywheel: Note, Nunes is transparently trying to [Trump] up a reason for Trump to fire these guys.
]
[NEUTRAL
This from my friend.
Hadrian Micciche
30 mins · 
You will not find sprin

[NEUTRAL
@Amy_Siskind: Remember when Yates was fired and we said this is the end. 
Remember when Comey was fired and we said this is the end. 
Ou…
]
[NEUTRAL
@joss Correction: Trump is REVIVING the country!
]
[NEUTRAL
@lord_inario: —Y dijo: Ya no vamos a comprar en el extranjero lo que consumimos. Todo lo vamos a producir aquí.
— Jajaja, ese Trump cada…
]
[NEUTRAL
@AbelMunizJr: Can't wait for the movie about these events starring Tom Hanks. https://t.co/F14PaseiqM
]
[NEUTRAL
@goodoldcatchy: The 30 years from 1987 (the vote to abolish the Fairness Doctrine) to 2017 will be seen by historians as the pre-Trump e…
]
[NEUTRAL
@krassenstein: Trump Supporter Lou Dobbs posted the following Twitter poll: 

Do you believe the corrupt leadership and actions of the D…
]
[NEUTRAL
@100_Disable_vet: It's a conspiracy between Hillary, Obama, Comey, Sessions, Deep State, Swamp, Rosenstein, Several federal judges, Muel…
]
[NEUTRAL
Teanna Trump is in my state and I'm just tryna get that girl to sit on my

[POSITIVE
@marklhaag: Good thread https://t.co/RSJeZrRBpg
]
[NEUTRAL
@AnnaSilPrad: The Darkness my instinct is going!!!! #POTUS  is TWO Steps ahead🇺🇸KEEP the FAITH : you know you’re gonna live thru the rai…
]
[NEUTRAL
@funder: @CNN @ErinBurnett Alan Dershowitz hung out with Trump recently at Mar-a-Lago-Anyone listening to him needs to stop #AMJoy #Trum…
]
[NEUTRAL
@USAHotLips: STOP &amp; BREATHE‼️

@realDonaldTrump isnt going anywhere. What we all should be concerned about are the methods #DeepState ar…
]
[NEUTRAL
#DeepState #QAnon Jack @POTUS #qanon4chan https://t.co/1dROgDSDuk
]
[NEUTRAL
@Navsteva: Russian Defense Ministry public advisory council member Igor Korotchenko: "Trump has to understand that we’re going to be tal…
]
[NEUTRAL
@starcrosswolf: A warning &amp; you better heed my words. If liberals take control of the House &amp; or Senate WE ARE DONE. Free speech GONE. T…
]
[NEUTRAL
@adamdavidson: Other than the President, nobody knows as much as Cohen. (And Cohen does seem to h

...

[NEUTRAL
@mmpadellan: I may not have agreed with many things Barbara Bush said, but the former First Lady never minced her words about trump.

I…
]
[NEUTRAL
@chrisvega25: 🌎IN A PERFECT WORLD🌍 

If none of these 🤡’s grew up “privileged”

Trump - would be in prison &amp; registered sex offender

Sa…
]
[NEUTRAL
@kylegriffin1: The White House kept it quiet, but Axios reports that Trump nearly blocked Mike Pence from getting his chosen national se…
]
[NEUTRAL
@Mmarty1230: Poll: Trump approval hits highest mark since first 100 days in office https://t.co/pc96tUEdsy via @realalexjones WAHOO WAHO…
]
[NEUTRAL
@brycetache: If you've followed me a while, you know optimism is my thing. I've always seen hope flickering in the distance. I've had mo…
]
[NEUTRAL
@sandy45_46: @Comey Jimbo all teed off cause President Trump gave you a chance to come clean and you blew it, you're fired was his only…
]
[NEUTRAL
@JudicialWatch: JW filed a FOIA lawsuit w/ the DOJ for info on any &amp; all communications &am

[NEUTRAL
@Reuters: France's Macron says he persuaded Trump to keep troops in Syria https://t.co/B3cApIch8s https://t.co/Y6SS3Kqp2y
]
[NEGATIVE
@MargaretJay7: @realDonaldTrump Yes trump, we spent a billion dollars bombing 2 empty buildings in Syria...great job loser!!!
]
[NEUTRAL
@StephenMilIer: Donald Trump is one of the only US Presidents to survive a coup. Liberals can whine like babies, but Trump is here to st…
]
[NEUTRAL
@allanbrauer: They were reportedly headed to DC for positions in the Trump administration. https://t.co/yUF2Vm0Sbr
]
[NEUTRAL
@andres20ad: Hace años EEUU atacó a Irak en base a una mentira, como lo eran las "armas de destrucción masiva" y se quedó con el petróle…
]
[NEUTRAL
@dtxErgaOmnes: .@BetoORourke would vote to impeach Trump, but right now he’s “focused on the issues” to beat Ted Cruz. #BetoForTexas 

h…
]
[NEUTRAL
#POTUS says raid on personal lawyer Cohen makes Attorney Client privilege I thing of the past https://t.co/BlYzKrRZBY
]
[NEUTRAL
Totally agree, tim

[NEUTRAL
@FoxNews @Franklin_Graham Trump is not religious, does not pray and has gone to church twice since he was inaugurated.
]
[NEUTRAL
@SethAbramson: 10/ Here's a link to the 79 pages of McCabe-Comey emails that Judicial Watch uncovered.

If you want to understand how Tr…
]
[NEUTRAL
@krassenstein: 18 Hours ago the White House Press Secretary, Sarah Sanders released a photo of Trump being briefed on Syria...

The Prob…
]
[NEUTRAL
@tedlieu: Today is Sunday. That means we all need to ask again: Why is Jared Kushner still a Senior White House Advisor? https://t.co/KA…
]
[NEUTRAL
@MSNBC: Rep. Maxine Waters: President Trump is perhaps "most deplorable human being" I've ever encountered https://t.co/2oH5fHhx4K
]
[NEUTRAL
@joncoopertweets: Nothing has changed, folks. Donald Trump is STILL a lying, immoral, bigoted charlatan. He’s STILL a xenophobe. He’s ST…
]
[POSITIVE
@Education4Libs: It makes me extremely grateful when I think about everything Trump gave up in order to run for President 

[NEUTRAL
@BethShell: @SusanDa50168535 @Alt_Spicerlies @renato_mariotti Help me out here...is that Dersh &amp; Graham with Leonard Blavatnik...Putin c…
]
[NEUTRAL
@BravenakBlog Also, mind the timing of the chemical attacks. Trump impulsively blurted out that we were getting out… https://t.co/goy9haTi1W
]
[NEUTRAL
@DTrumpPoll: Do you approve of @realDonaldTrump's performance as @POTUS?

#TheResistance #MAGA #Trump

48hrs to vote &amp; RT 😊
]
[NEUTRAL
@jdice03: “Perception is more important than reality.  If someone perceives something to be true, it is more important than if it is in…
]
[NEUTRAL
@C_3C_3: NYPD discovered Hillary’s emails on Weiner’s laptop during their underage sexting investigation.

McCabe, Priestap &amp; Toscas cov…
]
[NEUTRAL
@bakers_bread: Tell @POTUS Trump how he's doing,send him a message.Stop the CHEM TRAILS!! MASS ARRESTS!  #Qanon8chan #QAnon  #CBTS #TheS…
]
[NEUTRAL
@SethAbramson 1000x more credible than trump is baseline
]
[NEUTRAL
@mike_Zollo: If any other rep

[NEUTRAL
@AnneRiceAuthor: I'm struggling with "The Ballad of Stormy Daniels" set to a 19th century melody  in the public domain.  The rhymes are…
]
[NEUTRAL
@FoxNews: .@dangainor: Trump strikes Syria, media fire back, and other examples of their war against the president https://t.co/mMI2kFU7…
]
[NEUTRAL
@RockinHits: The Real Reason Evangelical "Christians" Don't Care About Trump's Sordid Sexual Behavior https://t.co/5hZZCkRIwg
]
[NEUTRAL
@tedlieu: Today is Sunday. That means we all need to ask again: Why is Jared Kushner still a Senior White House Advisor? https://t.co/KA…
]
[NEUTRAL
@RealEagleWings: .@KellyannePolls: "The president took action here that was not impulsive, and was very decisive."  Anyone who thinks Tr…
]
[NEUTRAL
@SusanStormXO: Dear @DineshDSouza 
I didn’t know you were still having to deal with this ISSUE ; Obama did . 
His ruthless Corrupt FBI C…
]
[NEUTRAL
@ElJeringasLoko: esperate a que veamos infinity war pinche trump
]
[NEUTRAL
@FMoniteau: #Ivanka is not qualifi

In [37]:
tweets_df.to_csv('./data/tweets_sentiment.txt')